<a href="https://colab.research.google.com/github/gibranhasan14/Side-Projects/blob/master/ARIMA_vs_XGBoost_(Bayesian%20Optimization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ARIMA MODEL FOR Sales PREDICTIONS


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import itemgetter
import scipy.stats as st
from statsmodels.tsa import stattools as stt
from statsmodels import tsa
import statsmodels.api as smapi
import datetime
from pandas.core import datetools
from statsmodels.tsa.stattools import adfuller
from colabtools import dremel
dateparse = lambda d: pd.datetime.strptime(d, '%Y/%m/%d')  

In [0]:
snapchat_billings.dtypes

In [0]:
snapchat_billings.set_index('scp_customer_entity_name')['gcp_billings']

In [0]:
def is_stationary(df, maxlag=12, autolag=None, regression='ct'):
    """Run the Augmented Dickey-Fuller test from statsmodels
    and print output.
    """
    outpt = stt.adfuller(df,maxlag=maxlag, autolag=autolag,
                            regression=regression)
    print('adf\t\t {0:.3f}'.format(outpt[0]))
    print('p\t\t {0:.3g}'.format(outpt[1]))
    print('crit. val.\t 1%: {0:.3f}, \
5%: {1:.3f}, 10%: {2:.3f}'.format(outpt[4]["1%"], 
                                     outpt[4]["5%"], outpt[4]["10%"]))
    print('stationary?\t {0}'.format(['true', 'false']\
                                   [outpt[0]>outpt[4]['5%']]))
    return outpt

In [0]:
def despine(axs):
    # to be able to handle subplot grids
    # it assumes the input is a list of 
    # axes instances, if it is not a list, 
    # it puts it in one
    if type(axs) != type([]):
        axs = [axs]
    for ax in axs:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.spines['bottom'].set_position(('outward', 10))
        ax.spines['left'].set_position(('outward', 10))

In [0]:
billings_series = pd.Series(snapchat_billings['gcp_billings'], index=snapchat_billings.index)

In [0]:
s_test = adfuller(billings_series, maxlag=12, autolag= None,regression='ct')
# extract p value from test results, try with autolag= 'aic'
print("p value > 0.05 means data is non-stationary: ", s_test[1])

In [0]:
snapchat= snapchat_billings.set_index('month_date')['gcp_billings']

In [0]:
snapchat.index

In [0]:
plt.plot(snapchat)
despine(plt.gca())
plt.gcf().autofmt_xdate()
plt.xlabel('booking_month')
plt.ylabel('Snapchat, Inc.')
plt.xlim('2016','2019')
plt.title('Snapchat Billings');

In [0]:
is_stationary(snapchat);


In [0]:
sc= snapchat_billings.set_index('month_date')['gcp_billings']
training, testing = train_test_split(sc, test_size=0.33, random_state=42)

In [0]:
testing

In [0]:
sc = training.copy()

# Step 1: Removing Stationarity- 

## Differencing

In [0]:
snapchat.diff(1).plot(label='1 period', title='Snapchat Billings') 
plt.legend(loc='best') 
despine(plt.gca())

In [0]:
is_stationary(snapchat.diff(1).dropna()) 

In [0]:
snapchat.diff(1).plot(label='1 period', title='Snapchat', 
                      dashes=(15,5)) 
snapchat.diff(1).diff(12).plot(label='1 and 12 period(s)', 
                               color='Coral') 
plt.legend(loc='best') 
despine(plt.gca()) 
plt.xlabel('Date')

In [0]:
is_stationary(snapchat.diff(1).diff(12).dropna()); 


In [0]:
is_stationary((snapchat-snapchat_seasonal).diff(1).dropna()); 


**Log Transformation**

In [0]:
log_series = np.log(snapchat)
log_series_shift = log_series - log_series.shift()
#log_series_shift = log_series_shift[∼np.isnan(log_series_shift)]

log_series_shift= log_series_shift.dropna()

is_stationary(log_series_shift);

In [0]:
snapchat.values

In [0]:
snapchat['z_data'] = (snapchat.values - snapchat.rolling(window=12).mean()) / snapchat.rolling(window=12).std()
snapchat['zp_data'] = snapchat['z_data'] - snapchat['z_data'].shift(12)

In [0]:
snapchat.index

In [0]:
adfuller(snapchat.z_data.dropna(), autolag='AIC', regression='ct')

In [0]:
adfuller(snapchat.zp_data.dropna(), autolag='AIC', regression='ct')

In [0]:
is_stationary(snapchat.zp_data.dropna(), autolag='AIC', regression='ct')

In [0]:
is_stationary(snapchat.z_data.dropna(), autolag='AIC', regression='ct')

In [0]:
sc= snapchat_billings.set_index('month_date')['gcp_billings']
sc_log = np.log(sc)
moving_avg = pd.rolling_mean(sc_log,12)
sc_log_moving_avg_diff = sc_log - moving_avg
sc_log_moving_avg_diff.dropna(inplace=True)

In [0]:
is_stationary(sc_log_moving_avg_diff)

In [0]:
test=check_stationary(sc_log_moving_avg_diff)

In [0]:
def check_stationary(df, maxlag=12, autolag='AIC', regression='ct'):
    """Run the Augmented Dickey-Fuller test from statsmodels
    and print output.
    """
    outpt = stt.adfuller(df, maxlag = maxlag, autolag=autolag, regression=regression)
    adf = outpt[0]
    p= outpt[1]
    crit_val= outpt[4]["1%"], outpt[4]["5%"], outpt[4]["10%"]
    stationary= ['true', 'false'][outpt[0]>outpt[4]['5%']]
    return adf, p, crit_val, stationary

In [0]:
def stationarize(snapchat_billings):
  sc= snapchat_billings.set_index('month_date')['gcp_billings']
  
  stat_dict ={}
  #Method 1:
  sc_log = np.log(sc)
  moving_avg = pd.rolling_mean(sc_log,12)
  sc_log_moving_avg_diff = sc_log - moving_avg
  sc_log_moving_avg_diff.dropna(inplace=True)
  test=check_stationary(sc_log_moving_avg_diff)
  p1= test[1]
  stat_dict.update( {'sc_log_moving_avg_diff' : p1} )
  
  #Method4 Decomposing:
  snapchat_decomp = seasonal_decompose(sc, freq=12)
  snapchat_trend = snapchat_decomp.trend
  snapchat_seasonal = snapchat_decomp.seasonal
  snapchat_residual = snapchat_decomp.resid
  snapchat_residual.dropna(inplace=True)
  test6=check_stationary(snapchat_residual.dropna());
  p6= test6[1]
  stat_dict.update( {'snapchat_residual' : p6} )
  
  #Method2 Differencing:
  test2= check_stationary(sc.diff(1).dropna()) 
  p2= test2[1]
  
  test3= check_stationary(sc.diff(1).diff(12).dropna()); 
  p3= test3[1]
  
  test4= check_stationary((sc-snapchat_seasonal).diff(1).dropna()); 
  p4= test4[1]
  
  #stat_dict.update( {'snapchat_residual' : p6} )
  stat_dict.update([ ('sc.diff(1)', p2) , ('sc.diff(1).diff(12)', p3) , ('(sc-snapchat_seasonal).diff(1)', p4)] )
  
  #Method3 Log Transformation (using shift):
  log_series = np.log(sc)
  log_series_shift = log_series - log_series.shift()
  log_series_shift= log_series_shift.dropna()
  test5 = check_stationary(log_series_shift)
  p5 = test5[1]
  
  stat_dict.update( {'log_series_shift' : p5} )
  
  #Method 5 Eliminating trend and seasonality: Seasonal Differencing
  sc_first_difference = sc - sc.shift(1) 
  test7= check_stationary(sc_first_difference.dropna(inplace=False))
  p7= test7[1]
  
  sc_seasonal_difference = sc - sc.shift(12)  
  test8= check_stationary(sc_seasonal_difference.dropna(inplace=False))
  p8= test8[1]
  
  sc_seasonal_first_difference = sc_first_difference - sc_first_difference.shift(12)  
  test9= check_stationary(sc_seasonal_first_difference.dropna(inplace=False))
  p9= test9[1]
  
  stat_dict.update([ ('sc_first_difference', p7) , ('sc_seasonal_difference', p8) , ('sc_seasonal_first_difference', p9)] )
  
  d= min(stat_dict.iteritems(), key=itemgetter(1))
  print('Best Method: {} p-value = {}'.format(d[0], d[1]))
  
  return eval(min(stat_dict.iteritems(), key=itemgetter(1))[0])
  
  

In [0]:
# methods = ['sc_log_moving_avg_diff','snapchat_residual', 'Differencing 1', \
#            'Differencing 1 & 12','Differencing sc-snapchat_seasonal',      \
#            'log_series_shift','sc_first_difference','sc_seasonal_difference']

# p_values= [1,2,3,4,5,6,7,8,9]

# for n in range(len(methods)):
#   print ('Method: {} achieved P-Value: {}').format(methods[n], p_values[n])

#   methods = ['sc_log_moving_avg_diff','snapchat_residual', 'Differencing 1', \
#              'Differencing 1 & 12','Differencing sc-snapchat_seasonal',      \
#              'log_series_shift','sc_first_difference','sc_seasonal_difference']
  
#   for p, m in zip([p1,p2,p3,p4,p5,p6,p7,p8,p9], methods):
#     print ('Method: {} achieved P-Value: {}').format(m,p)
#     if p.min():
#       print m,p
  
#   return min(p1,p2,p3,p4,p5,p6,p7,p8,p9)

In [0]:
stationarize(snapchat_billings)

In [0]:
from statsmodels.tsa.arima_process import arma_generate_sample
import statsmodels.api as sm
import numpy as np

In [0]:
train= (sc-snapchat_seasonal).diff(1).dropna()

In [0]:
res = sm.tsa.arma_order_select_ic(train, ic=['aic', 'bic'], trend='nc')
res.aic_min_order
res.bic_min_order

In [0]:
train=train.astype(np.int)

In [0]:
train

In [0]:
model = ARIMA(sc, order=(1,1,0))  
snapchat_model = model.fit()

snapchat_model.plot_predict(start='2016-12-01', end='2022-01-01', alpha=.10)
despine(plt.gca()) 
plt.xlabel('Year')
plt.legend(loc='upper left');

print(snapchat_model.aic, snapchat_model.bic)

snapchat_model.predict(start='2016-02-01', end='2020-01-01',typ='levels')

#model_1predictions= snapchat_model.predict(start='2016-12-01', end='2020-01-01',typ='levels')

# forecast_accuracy=(((((fullsales - model_1predictions) / fullsales) * 100).abs())-100).abs()
# forecast_accuracy['2016-12-01':'2018-06-01']

## Decomposing components

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose
snapchat_decomp = seasonal_decompose(snapchat, freq=12)

In [0]:
snapchat_trend = snapchat_decomp.trend
snapchat_seasonal = snapchat_decomp.seasonal
snapchat_residual = snapchat_decomp.resid

In [0]:
def change_plot(ax):
    despine(ax)
    ax.locator_params(axis='y', nbins=5)
    plt.setp(ax.get_xticklabels(), rotation=90, ha='center')

plt.figure(figsize=(9,4.5))

plt.subplot(221)
plt.plot(snapchat, color='Green')
change_plot(plt.gca())
plt.title('snapchat', color='Green')
xl = plt.xlim()
yl = plt.ylim()

plt.subplot(222)
plt.plot(snapchat.index,snapchat_trend, 
         color='Coral')
change_plot(plt.gca())
plt.title('Trend', color='Coral')
plt.gca().yaxis.tick_right()
plt.gca().yaxis.set_label_position("right")
plt.xlim(xl)
plt.ylim(yl)

plt.subplot(223)
plt.plot(snapchat.index,snapchat_seasonal, 
         color='SteelBlue')
change_plot(plt.gca())
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_major_formatter(plt.NullFormatter())
plt.xlabel('Seasonality', color='SteelBlue', labelpad=-20)
plt.xlim(xl)
plt.ylim((-8000,8000))

plt.subplot(224)
plt.plot(snapchat.index,snapchat_residual,
        color='IndianRed')
change_plot(plt.gca())
plt.xlim(xl)
plt.gca().yaxis.tick_right()
plt.gca().yaxis.set_label_position("right")
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_major_formatter(plt.NullFormatter())
plt.ylim((-8000,8000))
plt.xlabel('Residuals', color='IndianRed', labelpad=-20)

plt.tight_layout()
plt.subplots_adjust(hspace=0.55)

In [0]:
fig = plt.figure(figsize=(10,5) )

ax1 = fig.add_axes([0.1,0.1,0.6,0.9])
ax1.plot(snapchat-snapchat_trend, 
         color='Green', label='Detrended data')
ax1.plot(snapchat_seasonal, 
         color='Coral', label='Seasonal component')
kwrds=dict(lw=1.5, color='0.6', alpha=0.8)
d1 = pd.datetime(2015,1,1)
dd = pd.Timedelta('365 Days')
[ax1.axvline(d1+dd*i, dashes=(3,5),**kwrds) for i in range(9)]
d2 = pd.datetime(2015,3,1)
[ax1.axvline(d2+dd*i, dashes=(2,2),**kwrds) for i in range(9)]
ax1.set_ylim((-12000,10000))

ax1.locator_params(axis='y', nbins=4)
ax1.set_xlabel('Year')
ax1.set_title('snapchat Seasonality')
ax1.set_ylabel('snapchat')
ax1.legend(loc=0, ncol=2, frameon=True);

ax2 = fig.add_axes([0.8,0.1,0.4,0.9])
ax2.plot(snapchat_seasonal['2015':'2018'], 
         color='Coral', label='Seasonal component')
ax2.set_ylim((-12000,10000))
[ax2.axvline(d1+dd*i, dashes=(3,5),**kwrds) for i in range(1)]
d2 = pd.datetime(2015,3,1)
[ax2.axvline(d2+dd*i, dashes=(2,2),**kwrds) for i in range(1)]
despine([ax1, ax2])

import matplotlib.dates as mpldates
yrsfmt = mpldates.DateFormatter('%b')
ax2.xaxis.set_major_formatter(yrsfmt)
labels = ax2.get_xticklabels()
plt.setp(labels, rotation=90);

In [0]:
snapchat_seasonal_component = snapchat_seasonal['2015'].values
snapchat_seasonal_component

In [0]:
snapchat_residual.dropna(inplace=True)
is_stationary(snapchat_residual.dropna());

In [0]:
snapchat_residual

In [0]:
loc, shape = st.norm.fit(snapchat_residual)
x=range(-3000,3000)
y = st.norm.pdf(x, loc, shape)
n, bins, patches = plt.hist(snapchat_residual, bins=20, normed=True)
plt.plot(x,y, color='Coral')
despine(plt.gca())
plt.title('Residuals')
plt.xlabel('Value'); plt.ylabel('Counts');

In [0]:
(osm,osr), (slope, intercept, r) = st.probplot(snapchat_residual, dist='norm', fit=True)
line_func = lambda x: slope*x + intercept
plt.plot(osm,osr,
         '.', label='Data', color='Coral')
plt.plot(osm, line_func(osm), 
         color='SteelBlue',
         dashes=(20,5), label='Fit')
plt.xlabel('Quantiles'); plt.ylabel('Ordered Values')
despine(plt.gca())
plt.text(1, -14, 'R$^2$={0:.3f}'.format(r))
plt.title('Probability Plot')
plt.legend(loc='best', numpoints=4, handlelength=4);

In [0]:
snapchat.diff(1).plot(label='1 period', title='Snapchat Billings')
plt.legend(loc='best')
despine(plt.gca())

In [0]:
is_stationary(snapchat.diff(1).dropna());


In [0]:
sales.diff(1).plot(label='1 period', title='Partner sales',
                      dashes=(15,5))
sales.diff(1).diff(12).plot(label='1 and 12 period(s)',
                               color='Coral')
plt.legend(loc='best')
despine(plt.gca())
plt.xlabel('Date')

In [0]:
is_stationary(sales.diff(1).diff(12).dropna());


In [0]:
sales.diff(1).plot(label='1 period', title='Partner sales',
                      dashes=(15,5))
sales.diff(1).diff(12).plot(label='1 and 12 period(s)',
                               color='Coral')
plt.legend(loc='best')
despine(plt.gca())
plt.xlabel('Date')

In [0]:
is_stationary(sales.diff(1).diff(12).dropna());


# Time series Models

In [0]:
from statsmodels.tsa.arima_model import ARIMA


In [0]:
is_stationary((sales-sales_seasonal).diff(1).dropna());


In [0]:
ts = sales-sales_seasonal.dropna()
tsdiff = ts.diff(1)

In [0]:
ts

In [0]:
is_stationary((ts).dropna());


In [0]:
is_stationary(ts);


In [0]:
plt.plot(ts) # was tsdiff before this


In [0]:
model = ARIMA(ts, order=(3, 1, 1))  
arres = model.fit()

In [0]:
arres.plot_predict(start='2016-01-01', end='2018-08-01', alpha=0.10)
plt.legend(loc='upper left')
print(arres.aic, arres.bic)

In [0]:
model = ARIMA(ts, order=(1, 1, 1))  
mares = model.fit()

# AUTO SELECTING P , D , Q

In [0]:
from colabtools import adhoc_import

In [0]:
with adhoc_import.Google3():
  from google3.experimental.users.henrylee.py import more_ranklab_utils

In [0]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/wwwusage.csv', names=['value'], header=0)

model = pm.auto_arima(df.value, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

# Selecting p and q

In [0]:
tsa.stattools.arma_order_select_ic(tsdiff.dropna(), max_ar=3, max_ma=3, ic='aic')


In [0]:
tsa.stattools.arma_order_select_ic(tsdiff.dropna(), max_ar=3, max_ma=3, ic='bic')


# Selecting p and q using sales_residual.dropna()


In [0]:
tsa.stattools.arma_order_select_ic(ts, max_ar=3, max_ma=3, ic='aic')


In [0]:
tsa.stattools.arma_order_select_ic(ts, max_ar=3, max_ma=3, ic='bic')


# Selecting p and q using sales_residual.dropna()

In [0]:
tsa.stattools.arma_order_select_ic(sales_residual.dropna(), max_ar=3, max_ma=3, ic='aic')


In [0]:
tsa.stattools.arma_order_select_ic(sales_residual.dropna(), max_ar=3, max_ma=3, ic='bic')


In [0]:
acf = stt.acf(ts, nlags=5)
pacf = stt.pacf(ts, nlags=5)

In [0]:
y=-1.96/np.sqrt(len(ts))
y

In [0]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,2))
ax1.axhline(y=0,color='gray')
ax1.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax1.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax1.axvline(x=1,ls=':',color='gray')
ax1.plot(acf)
ax1.set_title('ACF')

ax2.axhline(y=0,color='gray')
ax2.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax2.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax2.axvline(x=1,ls=':',color='gray')
ax2.plot(pacf)
ax2.set_title('PACF')

despine([ax1,ax2])

In [0]:
ts

# Autoregressive Integrated Moving Average – ARIMA


In [0]:
fullsales = pd.read_excel('ML Trial.xlsx', sheet_name='Copy of ML TRIAL',
                   parse_dates=['booking_month'], 
                   index_col='booking_month', 
                   )

In [0]:
fullsales= fullsales['monthlybookings']

In [0]:
fullsales

In [0]:
model = ARIMA(ts, order=(0,1,1))  
arimares = model.fit()

In [0]:
arimares.plot_predict(start='2016-12-01', end='2020-01-01', alpha=.10)
plt.legend(loc='upper left');
print(arimares.aic, arimares.bic)

In [0]:
arimares.predict(start='2016-12-01', end='2020-01-01',typ='levels')

In [0]:
model_1predictions= arimares.predict(start='2016-12-01', end='2020-01-01',typ='levels')

In [0]:
forecast_accuracy=(((((fullsales - model_1predictions) / fullsales) * 100).abs())-100).abs()
forecast_accuracy['2016-12-01':'2018-06-01']

In [0]:
model = ARIMA(ts, order=(0,2,1))  
arimares = model.fit()

In [0]:
arimares.plot_predict(start='2016-12-01', end='2020-01-01', alpha=.10)
plt.legend(loc='upper left');
print(arimares.aic, arimares.bic)

In [0]:
arimares.predict(start='2016-12-01', end='2020-01-01',typ='levels')

In [0]:
model_2predictions= arimares.predict(start='2016-12-01', end='2020-01-01',typ='levels')

In [0]:
forecast_accuracy2=(((((fullsales - model_2predictions) / fullsales) * 100).abs())-100).abs()
forecast_accuracy2['2016-12-01':'2018-06-01']

In [0]:
fullsales

In [0]:
model = ARIMA(ts, order=(0,2,1))  
arimares = model.fit()

In [0]:
arimares.plot_predict(start='2016-12-01', end='2020-01-01', alpha=.10)
plt.legend(loc='upper left');
print(arimares.aic, arimares.bic)

In [0]:
arimares.predict(start='2016-12-01', end='2020-01-01',typ='levels')

In [0]:
model_3predictions=arimares.predict(start='2016-12-01', end='2020-01-01',typ='levels')

In [0]:
forecast_accuracy3=(((((fullsales - model_3predictions) / fullsales) * 100).abs())-100).abs()
forecast_accuracy3['2016-12-01':'2018-06-01']


In [0]:
forecast_accuracypercents= forecast_accuracy3['2016-12-01':'2018-06-01']
forecast_accuracypercents.shape

# Combined Model

In [0]:
model = ARIMA(ts_log.dropna(), order=(0, 1, 1))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))

In [0]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff.head()


In [0]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()

predictions_ARIMA_diff_cumsum

In [0]:
predictions_ARIMA_log = pd.Series(ts_log.ix[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log

In [0]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
predictions_ARIMA

In [0]:
np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts))

In [0]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(ts)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts)))

In [0]:
predictions_ARIMA

In [0]:
tsa.stattools.arma_order_select_ic(ts_log_diff.dropna(), max_ar=3, max_ma=3, ic='aic')


In [0]:
model = ARIMA(ts_log_diff, order=(0, 1, 1))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))

In [0]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff

In [0]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()

predictions_ARIMA_diff_cumsum.head()

In [0]:
predictions_ARIMA_log = pd.Series(ts_log.ix[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log

In [0]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
predictions_ARIMA

# XGBoost

In [0]:
!pip install bayesian-optimization

!pip install -U -q PyDrive

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost
from bayes_opt import BayesianOptimization
import numpy as np
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Sales

In [0]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('Sample - Superstore').sheet1
rows = worksheet.get_all_values()
df= pd.DataFrame.from_records(rows)

In [0]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header


#Expand feature list using Date Column
df['Order Date']= pd.to_datetime(df['Order Date'])
df['Month'] = df['Order Date'].dt.month
df['Year'] = df['Order Date'].dt.year
df['Quarter']= df['Order Date'].dt.quarter
df['Week']= df['Order Date'].dt.week
df['dayoftheweek']= df['Order Date'].dt.dayofweek

df = df.drop(['Quantity','Discount','Profit', 'Ship Date', 'Order Date'], axis=1)

In [0]:
df= df[df['Year']!=2014]

In [0]:
df.columns

In [0]:
df.loc[:, df.columns != 'Sales'].columns

In [0]:
sales= pd.get_dummies(data=df, columns= df.loc[:, df.columns != 'Sales'].columns)
sales['Sales']= pd.to_numeric(sales['Sales'])

In [0]:
#Separate target variable and features
X, y = sales.loc[:, sales.columns!='Sales'] , sales.loc[:, 'Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# encode string class values as integers
# label_encoder = LabelEncoder()
# label_encoder = label_encoder.fit(X['is_potential_fraud'])
# X.loc[:, 'is_potential_fraud'] = label_encoder.transform(X['is_potential_fraud'])

In [0]:
#MODEL #1 Very Basic - W/ No fine-tuning whatsoever

data_dmatrix = xgboost.DMatrix(data=X,label=y)
xg_reg = xgboost.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train, y_train)
preds1 = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds1))
print("RMSE: %f" % (rmse))

#pd.DataFrame(preds1)
#pd.Series(preds1)

(((((y_test - preds1) / y_test) * 100).abs())-100).abs(), \
SMAPE(preds1, y_test)

In [0]:
def cv_boost(x_train):
  """Use CV function to tune hyperparameters and run cross-validation on our training dataset and returns a mean MAE score"""
  dtrain = xgboost.DMatrix(x_train, label=y_train, feature_names=list(x_train.columns))

  params = {'max_depth':4, 'objective': 'reg:linear',
            'eta':0.085, 'gamma':10, 'silent':1, 'subsample':1}
  num_rounds = 1500
  cv_results = xgboost.cv(params, dtrain, num_boost_round=num_rounds, seed=42, nfold=5, metrics={'mae'}, early_stopping_rounds=10)
  return cv_results

cv_boost(X_train)

In [0]:
# USE CV TO FIND OPTIMAL max_depth and min_child_weight parameters

def cv_booster(x_train, y_train):
  
  dtrain = xgboost.DMatrix(x_train, label=y_train, feature_names=list(x_train.columns))
  params = {'max_depth':4, 'objective': 'reg:linear','eta':0.085, 'gamma':10, 'silent':1, 'subsample':1}
  num_rounds = 1500
  gridsearch_params = [ (max_depth, min_child_weight)
      for max_depth in range(9,12)
      for min_child_weight in range(5,8)]
  # Define initial best params and MAE
  min_mae = float("Inf")
  best_params = None
  for max_depth, min_child_weight in gridsearch_params:
    #print("CV with max_depth={}, min_child_weight={}".format(max_depth, min_child_weight))

    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgboost.cv(params, dtrain, num_boost_round=num_rounds, metrics={'mae'}, early_stopping_rounds=10)

    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].idxmin()
    #print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    
    if mean_mae < min_mae:
      min_mae = mean_mae
      best_params = (max_depth, min_child_weight)
  #print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))
  return best_params[0], best_params[1]

cv_booster(X_train, y_train)

In [0]:
#TESTING/ TWEAKING PARAMETERS

def xboost(x_train, y_train, x_test):
  """Trains xgboost model and returns predictions for x_test"""
  dtrain = xgboost.DMatrix(x_train, label=y_train, feature_names=list(x_train.columns))
  dtest = xgboost.DMatrix(x_test, feature_names=list(x_test.columns))

  params = {'max_depth':9, 'objective': 'reg:linear', #'eta':0.093 ----> 92.97% and SMAPE= 6.73
            'eta':0.199, 'gamma':10, 'min_child_weight':6,
            'silent':1, 'subsample':1 }
  
  """USE cv_booster function to get max_depth & min_child_weight"""
  params['max_depth'], params['min_child_weight'] = cv_booster(x_train, y_train) 
  
  num_rounds = 1500
  model = xgboost.train(params, dtrain, num_rounds)
  return pd.Series(model.predict(dtest))

##############################################################################
##############################################################################

def SMAPE(forecast, actual):
  """Returns the Symmetric Mean Absolute Percentage Error between two Series"""
  masked_arr = ~((forecast==0)&(actual==0))
  diff = abs(forecast[masked_arr] - actual[masked_arr])
  avg = (abs(forecast[masked_arr]) + abs(actual[masked_arr]))/2
  print('SMAPE Error Score: ' + str(round(sum(diff/avg)/len(forecast) * 100, 2)) + ' %')
  print(('Average Forecasting Accuracy: {} %').format(round((((((y_test - preds) / y_test) * 100).abs())-100).mean(), 2) * -1))

preds = xboost(X_train, y_train, X_test)
y_test= pd.Series(y_test, dtype='float32').reset_index(drop=True)

(((((y_test - preds) / y_test) * 100).abs())-100).abs().astype('float').round(2) \
, SMAPE(preds, y_test)

In [0]:
#Forecast Results (X_test, y_test, preds)
forecast_actual=pd.concat([X_test.reset_index(drop=True), pd.DataFrame(y_test), pd.DataFrame(preds)], axis=1).\
sort_values(by=['Year', 'Month', 'Quarter']).rename(columns={'gcp_billings':'actual_billings', 0:'Forecasted_billings'})
forecast_actual['Forecast_Accuracy']=(((((forecast_actual.actual_billings - forecast_actual.Forecasted_billings) / forecast_actual.actual_billings) * 100).abs())-100).abs()
forecast_actual['Forecast_Accuracy'] = forecast_actual['Forecast_Accuracy'].astype('float').round(2)
forecast_actual

In [0]:
pd.DataFrame(preds)

In [0]:
y_test

In [0]:
X_test

In [0]:
pd.DataFrame(xboost(X_train, y_train, X_test)).reset_index()

In [0]:
np.sqrt(mean_squared_error(preds, y_test))

## Bayesian Optimization

In [0]:
pip install jupyter_http_over_ws
# Optional: Install the extension to run every time the notebook server starts.
# Adds a /http_over_websocket endpoint to the Tornado notebook server.
jupyter serverextension enable --py jupyter_http_over_ws

In [0]:
pip install --upgrade jupyter_http_over_ws>=0.0.1a3 && \
  jupyter serverextension enable --py jupyter_http_over_ws

In [0]:
pip install jupyter_http_over_ws
jupyter serverextension enable --py jupyter_http_over_ws

In [0]:
!pip install jupyter_http_over_ws
# Optional: Install the extension to run every time the notebook server starts.
# Adds a /http_over_websocket endpoint to the Tornado notebook server.
!jupyter serverextension enable --py jupyter_http_over_ws

In [0]:
# !pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [0]:
# import h2o
# from h2o.estimators.gbm import H2OGradientBoostingEstimator

# h2o.init()
# h2o.connect()
# data = h2o.H2OFrame(sales)

# #data = h2o.upload_file(sales)
# train_cols = [x for x in data.col_names if x not in ['Sales']]
# target = "Sales"
# train, test = data.split_frame(ratios=[0.7])

In [0]:
def train_model(max_depth, 
                ntrees,
                min_rows, 
                learn_rate, 
                sample_rate, 
                col_sample_rate):
    params = {#'eval_metric': 'rmse',
        'objective': "reg:linear",
        'max_depth': int(max_depth),
        'ntrees': int(ntrees),
        'min_rows': int(min_rows),
        'learn_rate':learn_rate,
        'sample_rate':sample_rate,
        'col_sample_rate':col_sample_rate}
    
    dtrain = xgboost.DMatrix(X_train, label=y_train, feature_names=list(X_train.columns))
    #model = H2OGradientBoostingEstimator(nfolds=5,**params)
    num_rounds = 50
    
    cv_score = xgboost.cv(params, dtrain, num_boost_round=num_rounds, metrics={'rmse'}, early_stopping_rounds=10)
    #model = xgboost.train(params, dtrain, num_rounds)
    #model.train(x=X, y=y, training_frame=train)
    #return -model.rmse()
    return -1.0 * cv_score['test-rmse-mean'].iloc[-1]
  
  
bounds = {
    'max_depth':(5,10),
    'ntrees': (100,500),
    'min_rows':(10,30),
    'learn_rate':(0.001, 0.01),
    'sample_rate':(0.5,0.8),
    'col_sample_rate':(0.5,0.8)
}

In [0]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)
optimizer.maximize(init_points=10, n_iter=10, acq='ei')

In [0]:
optimizer.max

In [0]:
params = optimizer.res['max']['max_params']
params['max_depth'] = int(params['max_depth'])

In [0]:
"""Trains xgboost model and returns predictions for x_test"""

dtest = xgboost.DMatrix(X_test, feature_names=list(X_test.columns))
dtrain = xgboost.DMatrix(X_train, label=y_train, feature_names=list(X_train.columns))

params = optimizer.res['max']['max_params'] 

num_rounds = 1500
model = xgboost.train(params, dtrain, num_rounds)
return pd.Series(model.predict(dtest))


##############################################################################
##############################################################################

def SMAPE(forecast, actual):
  """Returns the Symmetric Mean Absolute Percentage Error between two Series"""
  masked_arr = ~((forecast==0)&(actual==0))
  diff = abs(forecast[masked_arr] - actual[masked_arr])
  avg = (abs(forecast[masked_arr]) + abs(actual[masked_arr]))/2
  print('SMAPE Error Score: ' + str(round(sum(diff/avg)/len(forecast) * 100, 2)) + ' %')
  print(('Average Forecasting Accuracy: {} %').format(round((((((y_test - preds) / y_test) * 100).abs())-100).mean(), 2) * -1))

#preds = xboost(X_train, y_train, X_test)
preds =pd.Series(model.predict(dtest))

y_test= pd.Series(y_test, dtype='float32').reset_index(drop=True)

(((((y_test - preds) / y_test) * 100).abs())-100).abs().astype('float').round(2) \
, SMAPE(preds, y_test)

## Prediction for Future Data

In [0]:
# List of series with future Dates, adding in 7 more rows(months)

listOfSeries = [pd.Series([None, 6, 2019, 2], index=sales.columns ) , pd.Series([None, 7, 2019, 3], index=sales.columns ) , 
                pd.Series([None, 8, 2019, 3], index=sales.columns ) , pd.Series([None, 9, 2019, 3], index=sales.columns ),
               pd.Series([None, 10, 2019, 4], index=sales.columns ),pd.Series([None,11, 2019, 4], index=sales.columns ),
               pd.Series([None, 12, 2019, 4], index=sales.columns )]

In [0]:
future_billings=sales.append(listOfSeries , ignore_index=True)

In [0]:
future_billings.tail(7)

In [0]:
#UNSEEN DATA
x_train = future_billings[pd.notnull(future_billings['gcp_billings'])].drop(['gcp_billings'], axis=1)
y_train = future_billings[pd.notnull(future_billings['gcp_billings'])]['gcp_billings']

# x_test goes into predict model, model trains on x_train while y is used as target variable in it.
x_test = future_billings[pd.isnull(future_billings['gcp_billings'])].drop(['gcp_billings'], axis=1)

In [0]:
future_preds=xboost(x_train, y_train, x_test)
pd.DataFrame(future_preds)

In [0]:
#7 Month sales Forecast
pd.concat([future_billings.tail(7).reset_index(drop=True), pd.DataFrame(future_preds)], axis=1).rename(columns={0:'Forecasted_billings'})

## k-fold Cross Validation using XGBoost


In [0]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'max_depth': 5, 'alpha': 10,'learning_rate': 0.1}

cv_results = xgboost.cv(dtrain=data_dmatrix, params=params, nfold=3, num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [0]:
cv_results.head()

print((cv_results["test-rmse-mean"]).tail(1))


In [0]:
cv_results

In [0]:
pd.DataFrame(preds)

In [0]:
xg_reg = xgboost.train(params=params, dtrain=data_dmatrix, num_boost_round=10)


In [0]:
import matplotlib.pyplot as plt

xgboost.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 10]
plt.show()

In [0]:
xgboost.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()